<a href="https://colab.research.google.com/github/Adityacodesinpython/UserPersonaProject/blob/main/UserPersonaApp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
pip install praw


In [8]:
import os

# Set secrets (you can paste your actual values here securely)
os.environ['REDDIT_CLIENT_ID'] = "Your_Client_Id"
os.environ['REDDIT_CLIENT_SECRET'] = "Your_Client_Secret"
os.environ['HUGGINGFACE_API_KEY'] = "Your_Api_Key"

In [9]:
import praw
from huggingface_hub import InferenceClient

# === Reddit API Setup ===
reddit = praw.Reddit(
    client_id=os.environ['REDDIT_CLIENT_ID'],
    client_secret=os.environ['REDDIT_CLIENT_SECRET'],
    user_agent="UserPersonaScraper"
)

client = InferenceClient(
    provider="auto",
    api_key=os.environ['HUGGINGFACE_API_KEY']
)

In [10]:
# === Extract username from URL ===
def extract_username(url):
    return url.rstrip('/').split('/')[-1]

# === Fetch user activity ===
def fetch_user_activity(username, limit=100):
    user = reddit.redditor(username)
    posts = [f"Post: {post.title}\n{post.selftext}" for post in user.submissions.new(limit=limit)]
    comments = [f"Comment: {comment.body}" for comment in user.comments.new(limit=limit)]
    return posts, comments

# === Generate structured user persona ===
def generate_advanced_persona(posts, comments):
    combined_data = posts + comments
    user_text = "\n\n".join(combined_data)

    prompt = f"""
You are a professional persona analyst.

Based on the following Reddit user's posts and comments, generate a **detailed user persona** with all the following fields. For each field, include 1–3 supporting quotes (citations) from the user's posts/comments to justify your inference.

Use this format:

Name: <choose a placeholder name>
Age: <e.g., 18–25>
Occupation:
Status: <e.g., student, working professional, etc.>
Location: <likely country or city>
Tier: <novice, expert, enthusiast>
Archetype: <e.g., The Explorer, The Helper, The Analyst>
Personality Type: <e.g., introvert, extrovert, practical, adaptable, etc.>
Motivations:
Behavior & Habits:
Frustrations:
Goals and Needs:

For each point, add supporting quotes from the text below like:
↳ "This is a direct quote from the user"

Reddit User Data:
{user_text}
"""

    response = client.chat.completions.create(
        model="mistralai/Mistral-7B-Instruct-v0.2",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=1400,
        temperature=0.7
    )

    return response.choices[0].message["content"]

# === Save output to uniquely named file ===
def save_to_file(text, username):
    filename = f"persona_u_{username}.txt"
    with open(filename, "w", encoding="utf-8") as f:
        f.write(f"👤 User Persona for u/{username}\n\n")
        f.write(text)
    print(f"Persona saved to {filename}")

In [11]:
# === Main Entry Point ===
if __name__ == "__main__":
    reddit_url = input("🔗 Enter Reddit user profile URL: ").strip()
    username = extract_username(reddit_url)

    print(f"📥 Fetching posts and comments for u/{username}...")
    posts, comments = fetch_user_activity(username)

    if not posts and not comments:
        print("⚠️ No user activity found.")
    else:
        print("🤖 Generating full user persona with citations...")
        persona = generate_advanced_persona(posts, comments)
        save_to_file(persona, username)

🔗 Enter Reddit user profile URL: https://www.reddit.com/user/Hungry-Move-6603/ 


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



📥 Fetching posts and comments for u/Hungry-Move-6603...


It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



🤖 Generating full user persona with citations...
Persona saved to persona_u_Hungry-Move-6603.txt
